# Ноутбук 01 — Подготовка данных и запуск скриптов

Цель: коротко показать, **какие скрипты запускаем** и **какие артефакты получаем**:
1) Осмотр сырья (`run_inspection.py`)
2) Очистка (`clean_data.py`) → `out/financial_clean.csv`, `out/prolongations_clean.csv`
3) Подготовка для SQL (`prepare_for_sql.py`) → `out/shipments.csv`, `out/projects_dim.csv`


## 1. Карта скриптов

Скрипты расположены в `src/` (в ряде версий — в `src/utils/`; ниже указываю оба варианта):

- `run_inspection.py` — первичный осмотр сырья (структура, кодировка, первые строки).
- `files_to_check.py` — параметры осмотра: какие файлы смотреть, ID-колонка, список кодировок, размер head.
- `inspector.py` — утилиты чтения и краткого описания таблиц.
- `clean_data.py` — очистка исходных `financial_data.csv` / `prolongations.csv`, нормализация чисел/текста.
- `prepare_for_sql.py` — подготовка набора для SQL: wide→long по месяцам, канонизация `YYYY-MM`,
  сборка факта `shipments.csv` и размерности `projects_dim.csv`.

## 2. `run_inspection.py` — первичный осмотр файлов

**Зачем:** убедиться, что сырье читается, увидеть кодировку, форму и первые строки.

**Читает:** список путей из `files_to_check.py` (`data/financial_data.csv`, `data/prolongations.csv`).

**Как работает (коротко):**
- Строит абсолютный путь к каждому файлу.
- Через `inspector.read_table(...)` пытается последовательно прочитать с несколькими кодировками.
- Через `inspector.summarize_df(...)` печатает форму, количество и пример колонок, `head`.

**Мини-фрагмент кода (для понимания):**
```python
# из run_inspection.py
from src.files_to_check import FILES, ID_COL, ENCODINGS, HEAD_ROWS
from src.utils.inspector import read_table, summarize_df

for path in FILES:
    p = abspath(path)
    df, enc, err = read_table(p, encodings=ENCODINGS, id_col=ID_COL)
    info = summarize_df(df, head_rows=HEAD_ROWS)
    # печатает: encoding, shape, columns_sample, head



### Ячейка 4 — files_to_check.py и inspector.py (параметры и утилиты)
```markdown
## 3. Конфигурация осмотра и утилиты

### `files_to_check.py`
- `FILES` — какие файлы проверяем (по умолчанию `data/financial_data.csv`, `data/prolongations.csv`).
- `ID_COL = "id"` — основная ключевая колонка, читается строкой.
- `ENCODINGS = ["utf-8-sig", "utf-8", "cp1251"]` — пробуем в таком порядке.
- `HEAD_ROWS = 5` — сколько строк показывать.

### `inspector.py`
- `read_table(path, encodings, id_col)` — надёжное чтение CSV/XLSX с перебором кодировок; `id` читается как строка.
- `summarize_df(df, head_rows)` — краткое резюме: форма, примеры колонок, head.

**Мини-фрагмент для ориентира:**
```python
# из inspector.py
DEFAULT_ENCODINGS = ["utf-8-sig", "utf-8", "cp1251"]

def read_table(path, encodings=None, id_col=None):
    # 1) проверяем наличие файла
    # 2) если Excel — читаем через pandas
    # 3) если CSV — перебираем кодировки; dtype для id -> string
    return df, "использованная_кодировка", {}

def summarize_df(df, head_rows=5):
    return {
        "shape": tuple(df.shape),
        "columns_count": df.shape[1],
        "columns_sample": list(df.columns[:min(10, df.shape[1])]),
        "head": df.head(head_rows).to_string(index=False)
    }



### Ячейка 5 — clean_data.py (очистка и нормализация)
```markdown
## 4. `clean_data.py` — очистка `financial_data` и `prolongations`

**Задачи:**
- Надёжно прочитать оба исходных CSV (перебор кодировок).
- Убрать BOM/пробелы в названиях столбцов, нормализовать текст.
- Привести все «месячные» столбцы к числам (убрать пробелы и неразрывные пробелы, заменить запятые на точки,
  удалить служебные слова вроде «стоп»).
- В пролонгациях — построить каноническое представление месяца (`month_canon`) по строковым значениям месяца.

**Ключевые моменты:**
- Поиск месячных колонок делается по наличию строки **2022/2023/2024** в имени.
- Числа чистятся через последовательные `.str.replace(...)` и `pd.to_numeric(errors="coerce")`.
- Результат сохраняется в `out/financial_clean.csv` и `out/prolongations_clean.csv` (UTF-8, без BOM).

**Мини-фрагменты:**
```python
# из clean_data.py: чтение с перебором кодировок
def read_csv(p):
    for e in ("utf-8-sig", "utf-8", "cp1251"):
        try:
            return pd.read_csv(p, encoding=e, dtype={"id":"string"}, low_memory=False)
        except UnicodeDecodeError:
            pass
    return pd.read_csv(p, dtype={"id":"string"}, low_memory=False)



### Ячейка 6 — prepare_for_sql.py (wide→long, артефакты для SQL)
```markdown
## 5. `prepare_for_sql.py` — подготовка для SQL

**Задачи:**
1. Очистить заголовки (BOM, хвостовые пробелы).
2. Развернуть «месячные» колонки `financial_clean` из wide-формата в long-формат (`id × month × shipment`).
3. Канонизировать месяц → `YYYY-MM` (например, «январь 2023» → `2023-01`).
4. Суммировать дубли (если одна отгрузка разбита на части).
5. Сформировать:
   - **Факт** `shipments.csv`: `id, month, shipment`
   - **Размерность** `projects_dim.csv`: `id, month_last, AM[, Account]`

**Мини-фрагменты (для ориентира):**
```python
# из prepare_for_sql.py: фиксация заголовков
def _fix_cols(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns=lambda c: c.replace("\ufeff", "").strip(), inplace=True)
    return df



### Ячейка 7 — Что получается на выходе этапа подготовки
```markdown
## 6. Артефакты после этапа подготовки

- `out/financial_clean.csv` — очищенные финданные; месяцы приведены к числам.
- `out/prolongations_clean.csv` — очищенные пролонгации; `month_canon` с форматом `YYYY-MM`.
- `out/shipments.csv` — «длинный» факт для SQL (id × месяц × сумма отгрузки).
- `out/projects_dim.csv` — размерность проекта (id × последний месяц пролонгации × AM × [Account]).

Эти файлы — **единственный вход** для SQL-агрегаций коэффициентов в ноутбуке 02.


## 8. Что дальше

В **Ноутбуке 02** фиксируем логику коэффициентов: определяем числитель/знаменатель, показываем SQL-запросы,
подгружаем `sql_results/*.csv`, делаем проверки корректности (диапазон [0..1], формат месяцев), и собираем таблицы
для `report.xlsx` (или сверяемся с уже готовым отчётом).
